<a href="https://colab.research.google.com/github/Sprakash058/plant-disease-detector/blob/main/plant_disease_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
train_labels = os.listdir('/gdrive/My Drive/train')
train_labels

['healthy', 'diseased']

In [ ]:
data = []
labels = []

for i in range(len(train_labels)):
    dir = os.path.join('/gdrive/My Drive/train', train_labels[i])
    images = os.listdir(dir)

    for x in range(len(images)):
        image = cv2.imread(dir + "/" + str(x+1) + ".jpg")
        image = cv2.resize(image, (500,500))
        image = np.array(image)
        data.append(image)
        labels.append(i)

In [ ]:
data = np.array(data)
labels = np.array(labels)

data.shape, labels.shape

((1709, 500, 500, 3), (1709,))

In [ ]:
#Splitting training and validation dataset
X_t, X_test, y_t, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_t, y_t, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((1093, 500, 500, 3),
 (342, 500, 500, 3),
 (274, 500, 500, 3),
 (1093,),
 (342,),
 (274,))

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPool2D(pool_size=(3, 3)),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(3, 3)),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(3, 3)),

    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(3, 3)),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(3, 3)),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(rate=0.8),

    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=40, epochs=8, validation_data=(X_val, y_val))

Epoch 1/8
28/28 [==============================] - 350s 12s/step - loss: 0.7884 - accuracy: 0.6020 - val_loss: 0.4755 - val_accuracy: 0.7774
Epoch 2/8
28/28 [==============================] - 343s 12s/step - loss: 0.4664 - accuracy: 0.8253 - val_loss: 0.4469 - val_accuracy: 0.8285
Epoch 3/8
28/28 [==============================] - 343s 12s/step - loss: 0.3774 - accuracy: 0.8481 - val_loss: 0.4840 - val_accuracy: 0.7847
Epoch 4/8
28/28 [==============================] - 340s 12s/step - loss: 0.3459 - accuracy: 0.8591 - val_loss: 0.2967 - val_accuracy: 0.8686
Epoch 5/8
28/28 [==============================] - 339s 12s/step - loss: 0.3161 - accuracy: 0.8664 - val_loss: 0.4287 - val_accuracy: 0.8431
Epoch 6/8
28/28 [==============================] - 336s 12s/step - loss: 0.2586 - accuracy: 0.8957 - val_loss: 0.3428 - val_accuracy: 0.8686
Epoch 7/8
28/28 [==============================] - 337s 12s/step - loss: 0.2455 - accuracy: 0.9012 - val_loss: 0.4019 - val_accuracy: 0.8467
Epoch 8/8
28/

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 498, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 166, 166, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 164, 164, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [ ]:
model.save("plant.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.load_model("plant.h5")

In [ ]:
pred = model.predict(X_test)
pred = pred.round()

11/11 [==============================] - 31s 3s/step


In [ ]:
accuracy_score(pred, y_test)

0.9619883040935673

In [ ]:
confusion_matrix(y_test, pred)

array([[180,   7],
       [  6, 149]])

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       187
           1       0.96      0.96      0.96       155

    accuracy                           0.96       342
   macro avg       0.96      0.96      0.96       342
weighted avg       0.96      0.96      0.96       342



In [ ]:
for i in range(10):
    # Generate a random integer between 0 and len(X_test)-1
    random_integer = tf.cast(tf.random.uniform(shape=(), minval=0, maxval=len(X_test)), tf.int32)
    print('Actual - ' + train_labels[y_test[random_integer]] + ', Predicted - ' + train_labels[int(pred[random_integer])])

Actual - diseased, Predicted - diseased
Actual - healthy, Predicted - healthy
Actual - diseased, Predicted - diseased
Actual - diseased, Predicted - diseased
Actual - healthy, Predicted - healthy
Actual - diseased, Predicted - diseased
Actual - diseased, Predicted - diseased
Actual - healthy, Predicted - healthy
Actual - healthy, Predicted - healthy
Actual - healthy, Predicted - healthy
